# <center>RPA transferencia entre bancos<center>

## Importações

In [184]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

## Pegando o login do banco apartir de um arquivo .ENV

In [185]:
load_dotenv()

db_host1 = os.getenv('DB_HOST1')
db_database1 = os.getenv('DB_DATABASE1')
db_user1 = os.getenv('DB_USER1')
db_password1 = os.getenv('DB_PASSWORD1')
db_port1 = os.getenv('DB_PORT1')

db_database2 = os.getenv('DB_DATABASE2')

print("DB_HOST1:", os.getenv('DB_HOST1'))
print("DB_DATABASE1:", os.getenv('DB_DATABASE1'))
print("DB_USER1:", os.getenv('DB_USER1'))
print("DB_PASSWORD1:", os.getenv('DB_PASSWORD1'))
print("DB_PORT1:", os.getenv('DB_PORT1'))
print("DB_DATABASE2:", os.getenv('DB_DATABASE2'))

DB_HOST1: recoop-germinare-9764.h.aivencloud.com
DB_DATABASE1: BANCO_1ANO
DB_USER1: avnadmin
DB_PASSWORD1: AVNS_DAFAJWqxMl1ba9hBbcZ
DB_PORT1: 16983
DB_DATABASE2: defaultdb


## Se conectando no banco do 1ano

In [186]:
print('Começando===============================================')
try:
    # Conectar ao banco de dados
    conn_1ano = psycopg2.connect(
        host=db_host1,
        database=db_database1,
        user=db_user1,
        password=db_password1,
        port=db_port1
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

Começando===============================================
Conexão estabelecida com sucesso!


## Pegando as informações do banco de dados

In [187]:
try:
    cursor_1ano = conn_1ano.cursor()

    cursor_1ano.execute('''SELECT * FROM Cooperativa''')
    results = cursor_1ano.fetchall()
    
    # Criar DataFrame
    columns = [desc[0] for desc in cursor_1ano.description]
    df = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Cooperativa''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_cooperativa = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Endereco''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_endereco = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Leilao''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_leilao = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Produto''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_produto = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Imagens''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_foto = pd.DataFrame(results, columns=columns)
    
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

## Se conectando no banco do 2ano

In [188]:
print('Começando===============================================')
try:
    # Conectar ao banco de dados
    conn = psycopg2.connect(
        host=db_host1,
        database=db_database2,
        user=db_user1,
        password=db_password1,
        port=db_port1
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

Começando===============================================
Conexão estabelecida com sucesso!


## Pegando as informações de login da cooperativa

In [189]:
display(df_cooperativa)

,cnpj,nome,email,senha,telefone,imagem,dados_status,status
0,99890123456709,Cooperativa novoYI,contato@cooperativai.com,senha@0157,67980127345,None,False,Ativo
1,44345128901234,Cooperativa novo,contato@cvmkvmkbooperativax.com,senha@1234,11987695432,None,False,Ativo
2,11916789455489,Cooperativa novoY,contato@coopermkbmbgativay.com,senha@5678,21987654432,None,False,Ativo
3,22265690123456,Cooperativa novoYC,contato@cooperabgbgtivac.com,senha@6745,34967895012,None,False,Ativo
4,33178909834586,Cooperativa novoYZ,contato@coobgbgperativaz.com,senha@9123,45978904123,None,False,Ativo
5,44289012345693,Cooperativa novoYD,contato@cooperbgbgativad.com,senha@7346,11989017234,None,False,Ativo
6,55890123456709,Cooperativa novoYI,contato@coopbgbgerativai.com,senha@0157,67980127345,None,False,Ativo
7,77916789455489,Cooperativa novoY,contato@coopermkbmbgativay.com,senha@5678,21987654432,None,False,Ativo
8,88265690123456,Cooperativa novoYC,contato@cooperabgbgtivac.com,senha@6745,34967895012,None,False,Ativo
9,99178909834586,Cooperativa novoYZ,contato@coobgbgperativaz.com,senha@9123,45978904123,None,False,Ativo


In [190]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()
                        
    for i in range(len(df_cooperativa)):
        if df_cooperativa['dados_status'][i] == True:
            try:
                # Tenta inserir no banco de dados
                cursor.execute('CALL insert_cooperativa(%s, %s, %s, %s, %s)', 
                    (df_cooperativa['cnpj'][i], df_cooperativa['nome'][i], df_cooperativa['email'][i], df_cooperativa['senha'][i], 
                    df_cooperativa['status'][i]))
            
            except Exception as e:
                print(f'Erro: {e}')
                print(f"Fazendo Update!!!===========================")
                conn.rollback()

                try:
                    # Tenta fazer o update
                    cursor.execute('UPDATE cooperativa SET nome_cooperativa=%s, email_cooperativa=%s, senha_cooperativa=%s where cnpj_cooperativa=%s',
                               (df_cooperativa['nome'][i], df_cooperativa['email'][i], df_cooperativa['senha'][i], df_cooperativa['cnpj'][i]))

                    # Confirma o update
                    conn.commit()

                except Exception as e_update:
                    conn.rollback()
                    print(f"Erro no update: {e_update}")

    conn.commit()
    print("Deu Green")

except Exception as e:
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        for i in range(len(df_cooperativa)):
            cursor_1ano.execute('UPDATE cooperativa SET dados_status=%s where cnpj=%s',
                                (False, df_cooperativa['cnpj'][i]))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")

# Leilão

In [191]:
display(df_leilao)

,id,data_inicio,data_fim,hora,valor_inicial,id_endereco,id_cooperativa,dados_status,status
0,1,2024-08-28,2024-09-01,12:50:00,1000.0,1,12345678901234,False,Ativo
1,2,2024-09-10,2024-09-15,12:51:00,2000.0,2,23456789065489,False,Ativo
2,3,2024-09-20,2025-01-10,12:52:00,3000.0,3,34567890123456,False,Ativo
3,4,2024-10-25,2024-10-27,12:53:00,4000.0,4,45678901234586,False,Ativo
4,5,2024-05-12,2024-06-02,12:54:00,5000.0,5,56789012345678,False,Ativo
5,6,2024-11-20,2024-12-17,12:55:00,6000.0,6,67890123456789,False,Ativo
6,8,2024-09-10,2024-09-15,12:51:00,2000.0,2,23456789065489,False,Ativo
7,9,2024-11-20,2024-12-17,12:55:00,6000.0,6,67890123456789,False,Ativo
8,7,2024-08-30,2024-09-01,12:50:00,1000.0,1,12345678901234,False,Ativo


In [192]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()

    for i in range(len(df_leilao)):
        if df_leilao['dados_status'][i] == True:  # Corrige o acesso à linha específica com [i]
            try:
                # Tenta inserir no banco de dados
                cursor.execute('CALL insert_leilao(%s, %s, %s, %s, %s, %s, %s)', 
                    (int(df_leilao['id_endereco'][i]), df_leilao['id_cooperativa'][i], df_leilao['data_inicio'][i], 
                     df_leilao['data_fim'][i], int(df_leilao['id'][i]), df_leilao['hora'][i], df_leilao['status'][i]))

                print('passei aqui')
            except Exception as e:
                print(f'Erro: {e}')
                print(f"Update feito==============")
                conn.rollback()

                try:
                    cursor.execute('UPDATE leilao SET data_inicio_leilao=%s, data_fim_leilao=%s, hora_leilao=%s, id_endereco=%s WHERE cnpj_cooperativa=%s', 
                    (df_leilao['data_inicio'][i], df_leilao['data_fim'][i], df_leilao['hora'][i], 
                     int(df_leilao['id_endereco'][i]), df_leilao['id_cooperativa'][i]))

                    conn_1ano.commit()
                except Exception as e:
                    print(f"Erro no finally ao tentar atualizar status: {e}")
                    conn.rollback()

    # Confirma as alterações
    conn.commit()
    print("Deu Green")

except Exception as e:
    # Faz rollback em caso de erro
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        for i in range(len(df_leilao)):
            cursor_1ano.execute('UPDATE leilao SET dados_status=%s WHERE id=%s', 
                                (False, int(df_leilao['id'][i])))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")

## Produto

In [193]:
display(df_produto)

,id,material,peso,id_leilao,dados_status,status
0,1,Plástico,500.0,1,False,Ativo
1,2,Metal,300.0,2,False,Ativo
2,3,Vidro,200.0,3,False,Ativo
3,4,Papel,700.0,4,False,Ativo
4,5,Metal,1000.0,5,False,Ativo
5,6,Papelao,100.0,6,False,Ativo


# AQUI

In [194]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()

    for i in range(len(df_produto)):
        if df_produto['dados_status'][i] == True:
            try:
                # Tenta inserir no banco de dados
                cursor.execute('''CALL insert_produto (%s, %s, %s, %s, %s, %s, %s)''', 
                    (int(df_produto['id'][i]), df_produto['material'][i], float(df_leilao['valor_inicial'][i]), 
                     float(df_produto['peso'][i]), df_foto['imagem'][i], int(df_produto['id_leilao'][i]), 
                     df_produto['status'][i]))
                
                # Confirma a inserção
                conn.commit()

            except Exception as e:
                # Faz rollback após erro no insert
                conn.rollback()
                print(f"Erro no insert: {e}")
                print("Fazendo Update================")
                
                try:
                    # Tenta fazer o update
                    cursor.execute('''UPDATE produto SET tipo_produto=%s, valor_inicial_produto=%s, peso_produto=%s, 
                                      foto_produto=%s, id_leilao=%s WHERE id_produto=%s''',
                                   (df_produto['material'][i], float(df_leilao['valor_inicial'][i]), 
                                    float(df_produto['peso'][i]), df_foto['imagem'][i], 
                                    int(df_produto['id_leilao'][i]), int(df_produto['id'][i])))

                    # Confirma o update
                    conn.commit()

                except Exception as e_update:
                    conn.rollback()
                    print(f"Erro no update: {e_update}")

    print("Deu Green")

except Exception as e:
    # Faz rollback se ocorrer erro geral
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        # Atualiza o campo dados_status na outra tabela
        for i in range(len(df_produto)):
            cursor_1ano.execute('UPDATE produto SET dados_status=%s WHERE id=%s',
                                (False, int(df_produto['id'][i])))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")

    finally:
        # Fecha os cursores e conexões
        cursor.close()
        conn.close()
        cursor_1ano.close()
        conn_1ano.close()


## Endereço

In [195]:
display(df_endereco)

,id,numero,complemento,cep,bairro,cidade,logradouro,estado,dados_status,status
0,2,456,Bloco A,87654-321,Bairro Alto,Rio de Janeiro,Rua Exemplo 2,RJ,False,Ativo
1,3,789,Casa,45678-123,Jardim,Belo Horizonte,Rua Exemplo 3,MG,False,Ativo
2,4,654,Apto 245,98453-745,Bairro do Arroz,Osasco,Rua Exemplo 4,SP,False,Ativo
3,6,467,Casa,78923-657,Jardim,Belo Horizonte,Rua Exemplo 6,MG,False,Ativo
4,7,123,Apto 101,12345-678,Centro,São Paulo,Rua Exemplo 1,SP,False,Ativo
5,8,456,Bloco A,87654-321,Bairro Alto,Rio de Janeiro,Rua Exemplo 2,RJ,False,Ativo
6,1,123,Apto 1023,12345-678,Centro,São Paulo,Rua Exemplo 1,SP,False,Ativo
7,5,912,Fundos,15489-452,Bairro C,Bairro CC,Rua Exemplo 5,PR,True,Ativo


In [196]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()

    for i in range(len(df_endereco)):
        if df_endereco['dados_status'][i] == True:  # Corrige a verificação do status
            id_endereco = int(df_endereco['id'][i])
            cidade = str(df_endereco['cidade'][i])
            logradouro = str(df_endereco['logradouro'][i])
            numero = int(df_endereco['numero'][i])
            status = str(df_endereco['status'][i])

            try:
                # Tenta inserir no banco de dados
                cursor.execute('CALL insert_endereco(%s::int, %s::varchar, %s::varchar, %s::int, %s::varchar)', 
                    (id_endereco, cidade, logradouro, numero, status))

            except Exception as e:
                # Caso o insert falhe, tenta fazer o update
                print(f"Falha ao inserir, tentando atualizar: {e}")
                conn.rollback()
                print('Update sendo feito============')
                try:
                    # Tenta fazer o update
                    cursor.execute('UPDATE endereco SET cidade=%s, rua=%s, numero=%s WHERE id_endereco=%s',
                    (cidade, logradouro, numero, id_endereco))  
                    
                    # Confirma o update
                    conn.commit()

                except Exception as e_update:
                    conn.rollback()
                    print(f"Erro no update: {e_update}")
                            

    # Confirma as alterações
    conn.commit()
    print("Deu Green")

except Exception as e:
    # Faz rollback em caso de erro
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        # Atualiza o campo dados_status na outra tabela
        for i in range(len(df_endereco)):
            id_endereco = int(df_endereco['id'][i])
            # Atualiza o status se a inserção for bem-sucedida
            cursor_1ano.execute('UPDATE endereco SET dados_status=%s WHERE id=%s', 
                            (False, id_endereco))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")
    # Fechar o cursor e a conexão
    cursor.close()
    conn.close()
    cursor_1ano.close()
    conn_1ano.close()

Falha ao inserir, tentando atualizar: duplicate key value violates unique constraint "endereco_pkey"
DETAIL:  Key (id_endereco)=(5) already exists.
CONTEXT:  SQL statement "INSERT INTO endereco (id_endereco,cidade, rua, numero, status) VALUES (en_id, en_cidade, en_rua, en_numero, en_status)"
PL/pgSQL function insert_endereco(integer,character varying,character varying,integer,character varying) line 4 at SQL statement

Update sendo feito============
Deu Green


In [197]:
cursor.close()
conn.close()
cursor_1ano.close()
conn_1ano.close()